# Android Dalvik虚拟机

## 一、简介
　　Dalvik是Google公司自己设计用于Android平台的虚拟机。它可以支持已转换为 .dex（即Dalvik Executable）格式的Java应用程序的运行，.dex格式是专为Dalvik设计的一种压缩格式，适合内存和处理器速度有限的系统。Dalvik 经过优化，允许在有限的内存中同时运行多个虚拟机的实例，并且每一个Dalvik 应用作为一个独立的Linux 进程执行。独立的进程可以防止在虚拟机崩溃的时候所有程序都被关闭。<br>
　　Dalvik和标准Java虚拟机(JVM)首要差别是：Dalvik 基于寄存器架构，而 JVM 基于栈。基于寄存器的虚拟机对于编译后变大的程序来说，在它们执行的时候，花费的时间更短。<br>
　　Dalvik和Java运行环境的区别：<br>
　　1、Dalvik主要是完成对象生命周期管理，堆栈管理，线程管理，安全和异常管理，以及垃圾回收等等重要功能。<br>
　　2、Dalvik负责进程隔离和线程管理，每一个Android应用在底层都会对应一个独立的Dalvik虚拟机实例，其代码在虚拟机的解释下得以执行。<br>
　　3、不同于Java虚拟机运行java字节码，Dalvik虚拟机运行的是其专有的文件格式Dex<br>
　　4、dex文件格式可以减少整体文件尺寸，提高I/o操作的类查找速度。<br>
　　5、odex是为了在运行过程中进一步提高性能，对dex文件的进一步优化。<br>
　　6、所有的Android应用的线程都对应一个Linux线程，虚拟机因而可以更多的依赖操作系统的线程调度和管理机制。<br>
　　7、有一个特殊的虚拟机进程Zygote，他是虚拟机实例的孵化器。它在系统启动的时候就会产生，它会完成虚拟机的初始化，库的加载，预制类库和初始化的操作。如果系统需要一个新的虚拟机实例，它会迅速复制自身，以最快的数据提供给系统。对于一些只读的系统库，所有虚拟机实例都和Zygote共享一块内存区域。<br>
　　本章将介绍Dalvik虚拟机特性与基于Dalvik字节码的汇编语言知识。<br>
## 二、Dalvik虚拟机特点——Android程序运行原理

### 2.1、Dalvik 虚拟机概述

### 2.2、Dalvik虚拟机与Java虚拟机的区别
#### 2.2.1、Java虚拟机运行的是Java字节码，Dalvik虚拟机运行的是Dalvik字节码
#### 2.2.2、Dalvik的可执行文件体积更小
#### 2.2.3、Java虚拟机与Dalvik虚拟机架构不同

### 2.3、Dalvik虚拟机是如何执行程序的

### 2.4、关于Dalvik虚拟机 JIT（即时编译、动态编译）
　　JIT，是一种通过在运行时将字节码编译为机器码的技术，可以使程序执行速度更快。<br>
　　主流的JIT包含两种字节码编译方式：<br>
　　1、method方式：以函数或方法为单位进行编译。<br>
　　2、trace方式：以trace为单位进行编译。<br>
　　程序中，一般很少是顺序执行代码的，多数代码都分成了好几条执行路径，其中函数的有些路径在实际运行过程中是很少被执行的，这部分路径被称为“冷路径”，而执行比较频繁的路径被称为“热路径”。采用传统 method 方式，以函数/方法为粒度会编译整个方法的代码，这样会在冷路径上浪费很多编译时间， 并且耗费更多的内存空间，却没有得到执行速度的提升。trace 方式编译，则能够快速的获取“热路径”代码，使用更短的时间与更少的内存来编译代码，等真的执行到冷路径时跳回到解释器或其它备用实行方式继续。<br>
　　

### 2.2、Dalvik汇编基础
　　Dalvik虚拟机为自己专门设计了一套指令集，并制定了自己的指令格式和调用规范。我们将Dalvik指令集组成的代码称为Dalvik汇编代码，由Dalvik汇编代码构成的程序文件即为smali文件（以.smali结尾,一般反编译而来，并非程序员编写，但可编辑后再编译）。<br>
### 2.2.1、smali文件头
　　smali文件的头3行:
>.class public Lcom/example/com/MainActivity;<br>
.super Ljava/lang/Object;<br> 
.source "MainActivity.java"<br>

　　说明：<br>
　　1、指定了当前类的类名,访问权限为public，类名开头的L是遵循Dalvik字节码的相关约定（Java对象）。<br>
　　2、super指令指定了当前类的父类，父类为Object。<br>
　　3、指定了当前类的源文件名。注意：经过混淆的dex文件，反编译出来的smali代码可能没有源文件信息，source可能为空。<br>
### 2.2.1、Dalvik指令格式
　　一段Dalvik汇编代码由一系列Dalvik指令组成，指令语法由指令的**位描述**与指令**格式标识**来决定。<br>
#### 2.2.1.1、指令的位描述
　　位描述约定如下：<br>
>1、每16位的字采用空格分隔开来。<br>
2、每个字母表示四位，每个字母按顺序从高字节开始，排列到低字节。每四位之间可能使用竖线“丨”来表示不同的内容。<br>
3、顺序采用 A〜Z 的单个大写字母作为一个4位的操作码，op 表示一个8位的操作码。<br>
4、“∅”来表示这字段所有位为0值。<br>

　　以指令格式“A|G|op BBBB F|E|D|C”为例：<br>
　　指令中间有两个空格，每个分开的部分大小为16位，所以这条指令由三个16位的字组成。第一个16位是“A|G|op”，高8位由A与G组成，低字节由操作码op组成。第二个16位由“BBBB”组成，它表示一个16位的偏移值。第三个16位分别由F、E、D、C共四个4 字节组成，在这里它们表示寄存器参数。<br>
#### 2.2.1.2、指令的格式标识
　　单独使用位标识还无法确定一条指令，必须通过指令格式标识来指定指令的格式编码。它的约定如下：<br>
>1、指令格式标识大多由三个字符组成，前两个是数字，最后一个是字母。<br>
2、第一个数字是表示指令有多少个16位的字组成。<br>
3、第二个数字是表示指令最多使用寄存器的个数。特殊标记“r”标识使用一定范围内的寄存器<br>
4、第三个字母为类型码，表示指令用到的额外数据的类型<br>

　　还有一种特殊情况，指令格式标识末尾可能会多出另一个字母，如果是字母“s”表示指令采用静态链接，如果是字母“i”表示指令应该被内联处理。<br>
#### 2.2.1.3、指令格式标识的类型码
|助记符|位大小|说明|
|:---|:---:|:---|
|b|8|8位有符号立即数|
|c|16,32|常量池索引|
|f|16|接口常量（仅对静态链接格式有效）|
|h|16|有符号立即数（32位数或64位数的高值位，低值位为0）|
|i|32|立即数，有符号整数或32位浮点数|
|l|64|立即数，有符号整数或64位双精度浮点数|
|m|16|方法常量（仅对静态链接格式有效）|
|n|4|4位的立即数|
|s|16|短整型立即数|
|t|8,16,32|跳转，分支|
|x|0|没有额外数据|

　　指令格式标识实例：22x<br>
　　第一个数字 2 表示指令有两个 16 位字组成，第二个数字 2 表示指令使用到 2 个寄存器， 第三个字母 x 表示没有使用到额外的数据。<br>

#### 2.2.1.4、Dalvik指令语法说明
　　Dalvik指令对语法约定如下：<br>
>1、每条指令从操作码开始，后面紧跟参数，参数个数不定，每个参数之间采用逗号分开。<br>
2、每条指令的参数从指令第一部分开始，op位于低8位，高8位可以是一个8位的 参数，也可以是两个4位的参数，还可以为空，如果指令超过16位，则后面部分依次作为参数。<br>
3、如果参数采用“vX”的方式表示，表明它是一个寄存器，如v0、v1等。这里采用 v 而不用 r 是为了避免与基于该虚拟机架构本身的寄存器命名产生冲突，如ARM 架构寄存器命名采用 r 开头。<br>
4、如果参数采用“#+X”的方式表示，表明它是一个常量数字。<br>
5、如果参数采用“+X”的方式表示，表明它是一个相对指令的地址偏移。<br>
6、如果参数采用“kind@X”的方式表示，表明它是一个常量池索引值。其中 kind 表示常量池类型，它可以是“string”（字符串常量池索引）、“type”（类型常量池索引）、“field”（字段常量池索引）或者“meth”（方法常量池索引）。<br>

　　指令实例：op vAA, string@BBBB
　　指令用到了 1个寄存器参数vAA，并且还附加了一个字符串常量池索引string@BBBB。其实这条指令格式代表着 const-string 指令。<br>
### 2.2.2、DEX文件反汇编工具
　　目前DEX可执行文件主流的反汇编工具有 **BakSmali** 与 **Dedexer** 。两种工具的反汇编代码大体的结构组织是一样的，在方法名、字段类型与代码指令序列上它们保持一致，具体的差异表现在一些语法细节上。<br>
　　BakSmali 提供反汇编功能的同时，还支持使用 Smali 工具打包反汇编代码重新生成 dex 文件，该功能被广泛应用于 apk 文件的修改、补丁、破解等场合，因而更加受到开发人员的青睐。<br>

### 2.2.3、了解Dalvik寄存器


### 2.2.4、寄存器表示法— v 命名法与 p 命名法
　　假设一个函数使用到M个寄存器，并且该函数有N个参数，根据 Dalvik 虚拟机参数传递方式中的规定：参数使用最后的N个寄存器，局部变量使用从v0开始的前M- N个寄存器。如前面小节中，foo() 函数使用到了5个寄存器，2个显式的整形参数，其中foo()函数是 Hello类的非静态方法，函数被调用时回传入一个隐式的Hello对象引用，因此，世纪传入的参数是3个。根据传参规则，局部变量将使用前2个寄存器，参数会使用后3个寄存器。<br>
　　**v命名法：**<br>
　　v命名方法采用小写字母“v”开头的方式表示函数中用到局部变量与参数，所有的寄存器命名从v0开始，依次递增。对于foo()函数，v命名法会用到v0,v1,v2,v3,v4等五个寄存器。v0与v1表示函数的局部变量寄存器，v2表示被传入的Hello对象的引用，v3与v4分别表示传入的整形参数。<br>
　　**p命名法**<br>
　　p命名方法对函数的局部变量的命名没有影响，它的命名规则是：函数中引入的参数从p0开始，依次递增。对于foo()函数，v命名法会用到v0,v1,p0,p1,p2等五个寄存器。v0与v1同样用来表示函数的局部变量寄存器，p0表示被传入的Hello对象的引用，p1与p2分别表示传入的整形参数。<br>
　　v命名法 与 p命名法对比表：<br>
  
  |v命名法|p命名法|寄存器含义|
  |:---:|:---:|:---|
  |v0| v0| 第一个局部变量寄存器|
  |v1| v1| 第二个局部变量寄存器|
  |...| ...| 中间的局部变量寄存器依次递增且名称相同|
  |vM-N| p0| 第一个参数寄存器|
  |...| ...| 中间的参数寄存器分别依次递增|
  |vM-1| pN-1| 第N个参数寄存器|

### 2.2.5、Dalvik 字节码的类型、方法与字段表示方法
　　Dalvik 字节码有着一套自己的类型、方法与字段表示方法，这些方法与Dalvik虚拟机指令集一起组成了一条条的Dalvik汇编代码。<br>
#### 2.2.5.1、Dalvik 类型
　　Dalvik字节码只有两种类型，基本类型与引用类型。Dalvik使用这两种类型来表示Java语言的全部类型，处了对象和数组属于引用类型外，其他的Java类型都是基本类型。
　　**Dalvik 字节码类型描述符表**

|Smali类型描述符|Java类型|备注|
|:---:|:---:|:---|
|v|void|只能用于返回值类型|
|Z|boolean||
|B|byte||
|S|short||
|C|char||
|I| int||
|J| long|64位类型|
|F|float||
|D|double|64位类型|
|Lpackage/name/ObjectName;|Java 类类型|L表示是一个对象类型，后面跟着一个java类，package/name表示该对象所在的包，<br>ObjectName表示对象的名称，“;”表示对象名称结束|
|\[类型|数组类型|如:\[I 表示一个int型数组；\[Ljava/long/String 表示一个String的对象数组|
|\[\[ 或 \[\[\[| 多维数组|最大克表示为数为255个，如：\[\[I 表示int[][]，\[\[\[I 表示int[][][]|

#### 2.2.5.2、Dalvik 方法
　　方法的表现形式比类名要复杂一些，Dalvik使用方法名、类型参数与返回值来详细描述一个方法。这样做一方面有助于Dalvik虚拟机在运行时从方法表中快速地找到正确的方法，另一方面，Dalvik虚拟机也可以使用它们来做一些静态分析，比如Dalvik字节码的验证与优化。方法格式如下：<br>
>Lpackage/name/ObjectName;->Methodname(III)Z

　　在这个例子中，Lpackage/name/ObjectName;应该理解为一个类型，MethodName为具体的方法名，(III)Z是方法的签名部分，其中括号内的III为方法的参数（在此为三个整型参数），Z表示方法的返回类型（boolean类型）。<br>
　　下面是一个更为复杂的例子：<br>
>method(I[[IILjava/lang/String;[Ljava/lang/Object;)Ljava/lang/String;

　　按照上面的知识，将其转换成Java形式的代码应该为：<br>
>String method(int, int[][], int, String, Object[])

　　BakSmali生成的方法代码 .method指令开始，以 .end method指令结束，根据方法类型的不同，在方法指令开始前可能会用“\#”号加以注释。如“\# virtual methods”表示这是一个虚方法，“\# direct methods”表示这是一个直接方法。<br>

#### 2.2.5.3、Dalvik 字段
　　字段与方法很相似，只是字段没有方法签名域中的参数与返回值，取而代之的是字段的类型。同样，Dalvik虚拟机定位字段与字节码静态分析时会用到它。字段的格式如下：<br>
>Lpackage/name/ObjectName;->FieldName:Ljava/lang/String;

　　字段由类型（Lpackage/name/ObjectName;）、字段名（FieldName）与字段类型（Ljava/lang/String;）组成。其中字段名与字段类型中间用冒号“:”隔开。<br>
　　BakSmali生成的字段代码以 .field指令开头，根据字段类型的不同，在字段指令的开始可能会用“\#”号加以注释，如：“\# instance fields”表示这是一个实例字段，“\# static fields”表示这是一个静态字段。<br>

## 2.3、Dalvik 指令集









